#### 訓練模型並輸出

## Include Library

In [1]:
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data
import argparse

C:\Users\shliaoc\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Train Process

In [10]:
def _main(annotation_path, classes_path, output_model_path):
    # return
    annotation_path = annotation_path
    log_dir = 'logs/000/'
    classes_path = classes_path

    class_names = get_classes(classes_path)
    num_classes = len(class_names)

    print(num_classes)
    anchors = get_anchors(anchors_path)
    

    input_shape = (416,416) # multiple of 32, hw

    is_tiny_version = len(anchors)==6 # default setting
    if is_tiny_version:
        model = create_tiny_model(input_shape, anchors, num_classes,
            freeze_body=2, weights_path='model/tiny_yolo_weights.h5')
    else:
        model = create_model(input_shape, anchors, num_classes,
            freeze_body=2, weights_path='model/yolo_weights.h5') # make sure you know what you freeze   
    # model.save('yolo_model_retrain.h5')  # creates a HDF5 file 'my_model.h5'

    print(model.input)
    print(model.output)
    #plot_model(model, to_file='model_data/retrained_model.png', show_shapes = True)

    logging = TensorBoard(log_dir=log_dir)
    #checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
    #      monitor='loss', save_weights_only=True, save_best_only=False, period=5)
    checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}.h5',
          monitor='loss', save_weights_only=True, save_best_only=False, period=5)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

    val_split = 0.00000001
    with open(annotation_path) as f:
        lines = f.readlines()
    np.random.seed(10101)
    np.random.shuffle(lines)
    np.random.seed(None)
    num_val = int(len(lines)*val_split)
    num_train = len(lines) - num_val
    
    # Unfreeze and continue training, to fine-tune.
    # Train longer if the result is not good.
    if True:
        for i in range(len(model.layers)):
            model.layers[i].trainable = True
        model.compile(optimizer=Adam(lr=1e-4), loss='mean_squared_error') # recompile to apply the change
        print('Unfreeze all of the layers.')

        batch_size = 8 # note that more GPU memory is required after unfreezing the body
        print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
        model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            #validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
            #validation_steps=max(1, num_val//batch_size),
            epochs=50,   #loss很大時先不要設太大
            initial_epoch=0,
            callbacks=[logging, checkpoint])
            #callbacks=[logging, checkpoint, reduce_lr, early_stopping])
        # model.save_weights(log_dir + 'trained_weights_final.h5')
        # model.save(log_dir + 'trained_model_final.h5')

    # Further training if needed.
    

    # print('model.input = ',model.input)
    # print('len(model.layers) = ',len(model.layers))
    # print('model.layers[-1]: ',model.layers[-1].output)
    # print('model.layers[-2]: ',model.layers[-2].output)
    # print('model.layers[-3]: ',model.layers[-3].output)
    # print('model.layers[-4]: ',model.layers[-4].output,'\n')
    # # original yolo model outputs:
    # print('model.layers[-5]: ',model.layers[-5].output)
    # print('model.layers[-6]: ',model.layers[-6].output)
    # print('model.layers[-7]: ',model.layers[-7].output)

    # save the derived model for detection(using yolo_video.py)
    derived_model = Model(model.input[0], [model.layers[249].output, model.layers[250].output, model.layers[251].output])
    #plot_model(derived_model, to_file=output_model_path[:-3]+'.png', show_shapes = True)
    derived_model.save(output_model_path)

## Subroutine

In [11]:
def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

In [12]:
def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

In [13]:
def create_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/yolo_weights.h5'):    #weight_path
    '''create the training model'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    # y_true = [Input(shape=(416//{0:32, 1:16, 2:8}[l], 416//{0:32, 1:16, 2:8}[l], 9//3, 80+5)) for l in range(3)]
    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], num_anchors//3, num_classes+5)) for l in range(3)]

    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)
    print('model_body.input: ', model_body.input)
    print('model.input: ', model.input)

    return model

In [14]:
def create_tiny_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/tiny_yolo_weights.h5'):
    '''create the training model, for Tiny YOLOv3'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16}[l], w//{0:32, 1:16}[l], \
        num_anchors//2, num_classes+5)) for l in range(2)]

    model_body = tiny_yolo_body(image_input, num_anchors//2, num_classes)
    print('Create Tiny YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze the darknet body or freeze all but 2 output layers.
            num = (20, len(model_body.layers)-2)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.7})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

In [15]:
def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)   # input of original yolo: image
        box_data = np.array(box_data)       # output of original yolo: boxes
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes) # some kind of output description?!
        yield [image_data, *y_true], np.zeros(batch_size)

In [16]:
def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)


##  Main Function

In [9]:

if __name__ == '__main__':
    
    anchors_path = 'model/yolo_anchors.txt'
    
    # For Path message
    annotation_path = "dataset/raccoon_train_data.txt"   # 候選框 TXT檔 位置
    classes_path = "dataset/box_classes.txt"             # 類別標籤位置
    output_model_path = "model/20210707_Test_ShengYe_Test_model.h5"       # 輸出的 model 要放的位置
    
    print('annotation_path = ', annotation_path)
    print('classes_path = ', classes_path)
    print('output_model_path = ', output_model_path)
    
    _main(annotation_path, classes_path, output_model_path)
    

annotation_path =  dataset/raccoon_train_data.txt
classes_path =  dataset/box_classes.txt
output_model_path =  model/20210707_Test_ShengYe_Test_model.h5
4
Create YOLOv3 model with 9 anchors and 4 classes.


OSError: Unable to open file (unable to open file: name = 'model/yolo_weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
'''
if __name__ == '__main__':
    
    parser = argparse.ArgumentParser()
    
    parser.add_argument("-a", "--annotation_path", type=str, default='test_data/training_data/annotation.txt', help="input annotation_path")
    parser.add_argument("-c", "--classes_path", type=str, default='test_data/training_data/pedestrian_classes.txt', help="input classes_path")
    parser.add_argument("-o", "--output_model_path", type=str, default='model_data/pedestrian_model.h5', help="input output_model_path")
    args = parser.parse_args()
    print('annotation_path = ', args.annotation_path)
    print('classes_path = ', args.classes_path)
    print('output_model_path = ', args.output_model_path)

    _main(args.annotation_path, args.classes_path, args.output_model_path)
'''